## Contamination modelling and detection
### Outline

- How to simulate a contamination event
- Prepare the simulations
- Contamination scenario
- Simulation the even with MSX
- Interval-based contamination detection 
- Contamination detection

### Initialize EPANET Python Toolkit (EPyT)
#### You should always begin with this command to import the toolkit: `from epyt import epanet`

In [ ]:
%pip install epyt

In [ ]:
from epyt import epanet

### How to simulate a contamination event
![Description](images/im1.png)



#### Create an arsenite contamination event

In [ ]:
Gc = epanet('data/BWSN_Network_1.inp', loadfile=True)   # Load EPANET Input file
Gmsx = Gc.loadMSXFile('data/Arsenite.msx')  # Load MSX File

##### Arsenite (As3) reacts with Chlorine (Cl2) and produces Arsenate (As5)

#### Prepare the simulations

In [ ]:
# Sensor locations
sensor_id = ['JUNCTION-17', 'JUNCTION-83', 'JUNCTION-122', 'JUNCTION-31', 'JUNCTION-45']
sensor_index = Gc.getNodeIndex(sensor_id)
Gc.plot(highlightnode=sensor_index)
Gc.plot_close() # Close - duplicate plots in jupyter

print(sensor_index)

In [ ]:
# Simulation Setup
t_d = 5 # days
Gc.setTimeSimulationDuration(t_d*24*60*60)  #Set simulation duration of 5 days
Gc.setMSXTimeStep(3600) 

### Contamination scenario
#### A contamination scenario needs a location (node), a contaminant concentration and a time profile (start and end time)

In [ ]:
injection_node = 'JUNCTION-23'
print('Injection node =', injection_node)

In [ ]:
injection_conc = 0.05
print('Injection concentration =', injection_conc)

In [ ]:
injection_start_time = 150
print('Injection start time =', injection_start_time)

In [ ]:
Gc.plot(highlightnode=injection_node);

### Simulation the event with MSX

#### Initialize the contamination vector (binary)

In [ ]:
# Import numpy package
import numpy as np

# Initialize the vector
as3_pat = np.zeros(t_d*48)  

# Create the injection pattern 
as3_pat[injection_start_time:] = 1  

print(as3_pat)

#### Simulate the contamination of water with arsenite

In [ ]:
# Specify Arsenite injection source
Gc.setMSXSources(injection_node, 'AsIII', 'SETPOINT', injection_conc, 'AS3PAT')

# Set pattern for injection
Gc.setMSXPattern('AS3PAT',  as3_pat.tolist()) 

# Solve hydraulics and MSX quality dynamics
Qmsx = Gc.getMSXComputedQualityNode(sensor_index)
print(Qmsx.Quality[18][:,0])

#### Plot the results

In [ ]:
import matplotlib.pyplot as plt

times = Qmsx.Time / 3600
for i, index in enumerate(sensor_index):
    fig, ax = plt.subplots()
    fig.set_figwidth(7)
    
    plt.plot(times, Qmsx.Quality[index][:,0], label='Cl2 (Chlorine)')
    plt.plot(times, Qmsx.Quality[index][:,1], label='As3 (Arsenate)')
    plt.plot(times, Qmsx.Quality[index][:,2], label='As5 (Arsenate)')
    
    plt.legend(loc='upper right', fontsize=7)
    plt.title(sensor_id[i], fontsize=8)
    plt.xlabel('Time (hours)', fontsize=8)
    plt.ylabel('Quality', fontsize=8)
    plt.show()

##### Unload library

In [ ]:
Gc.unloadMSX()
Gc.unload()

### Interval-based contamination detection
#### Initialize the simulation

In [ ]:
# Load EPANET Network
Gc = epanet('data/BWSN_Network_1.inp')   

In [ ]:
# Load MSX file
Gc.loadMSXFile('data/Arsenite.msx')  
 
# Sensor locations
sensor_id = ['JUNCTION-17', 'JUNCTION-83', 'JUNCTION-122', 'JUNCTION-31', 'JUNCTION-45']
print('sensor_id =', sensor_id)

In [ ]:
sensor_index = Gc.getNodeIndex(sensor_id)
print(sensor_index) 

In [ ]:
# Set simulation duration for 5 days
t_d = 5 # days
Gc.setTimeSimulationDuration(t_d*24*60*60)  

# Get Network data
demand_pattern = Gc.getPattern()
roughness_coeff = Gc.getLinkRoughnessCoeff()
node_id = Gc.getNodeNameID()

Gc.setMSXTimeStep(3600) # in seconds

#### Setup incertainties

In [ ]:
# Scenarios
Ns = 20 # Number of scenarios to simulate
u_p = 0.20  # Pattern uncertainty
Gc.printv(u_p) # Print value

In [ ]:
u_r = 0.20  # Roughness coefficient uncertainty
Gc.printv(u_r) # Print value

#### Create the scenarios - first without contamination, to compute the bounds

In [ ]:
max_inj_conc = 0.0
Gc.printv(max_inj_conc)

In [ ]:
# After day 2 (Dt = 30min)
inj_start_time = 2*48   
Gc.printv(inj_start_time)

In [ ]:
# Maximum duration of 12 hours
inj_duration = 24   
Gc.printv(inj_duration)

In [ ]:
inj_sc = np.column_stack([
    np.random.randint(1, Gc.NodeCount + 1, size=Ns),     # Injection locations
    max_inj_conc * np.random.rand(Ns),                   # Magnitudes
    np.random.randint(1, 49, size=Ns) + inj_start_time,  # Start times
    np.random.randint(1, inj_duration + 1, size=Ns)      # Durations
])
Gc.printv(inj_sc)

#### Run epochs

In [ ]:
QQ = []
for i in range(Ns):
    print('Iteration', i + 1)
    
    # Randomize demands
    r_p = -u_p + 2 * u_p * np.random.rand(*demand_pattern.shape)
    new_demand_pattern = demand_pattern + demand_pattern * r_p
    Gc.patterns = new_demand_pattern  # Set new patterns
    
    # Randomize pipe roughness
    r_r = -u_r + 2 * u_r * np.random.rand(*roughness_coeff.shape)
    new_roughness_coeff = roughness_coeff + roughness_coeff * r_r
    Gc.setLinkRoughnessCoeff(new_roughness_coeff) # Set new roughness coefficients
    
    # Simulate quality with randomized hydraulic parameters
    QQ.append(Gc.getMSXComputedQualityNode(sensor_index))  # Solve hydraulics and MSX quality dynamics
    

#### Plot the results without contaminations

In [ ]:
# Create subplots
fig, axs = plt.subplots(len(sensor_index), 1, figsize=(10, 15))

# Iterate over simulations and sensor indices
Qtable = []
for i in range(Ns):
    for j, sensor in enumerate(sensor_index):
        # Plot chlorine concentration for each sensor
        quality = QQ[i-1].Quality[sensor][:, 0]
        axs[j].plot(QQ[i-1].Time / 24 / 60 / 60, quality, '-', color=[0, 0.7, 0.9])
        axs[j].grid(True)
        axs[j].set_ylabel('Cl_2 (mg/L)')
        axs[j].set_xlabel('Time (days)')
        Qtable.append(quality)

# Set titles for each subplot
for i, ax in enumerate(axs):
    ax.set_title(sensor_id[i])

plt.tight_layout()
plt.show()

#### Compute the intervals for each sensor

In [ ]:
# Initialize upper and lower bound arrays
sensor_len = len(sensor_index)
UBQ = np.zeros((len(Qtable[0]), sensor_len))
LBQ = np.zeros((len(Qtable[0]), sensor_len))

# Iterate over sensor indices
for j in range(len(sensor_index)):
    # Calculate upper bound
    UBQ[:, j] = np.max(Qtable[j], axis=0)
    
    # Calculate lower bound
    LBQ[:, j] = np.min(Qtable[j], axis=0)

#### Plot the intervals

In [ ]:
Qmsx.Quality

In [ ]:
# Create subplots
fig, axs = plt.subplots(len(sensor_index), 1, figsize=(10, 15))

# Iterate over sensor indices
for j, sensor in enumerate(sensor_index):
    # Plot upper bound
    axs[j].plot(QQ[i].Time / 24 / 60 / 60, UBQ[:, j], 'r-', label='Upper Bound')
    # Plot lower bound
    axs[j].plot(QQ[i].Time / 24 / 60 / 60, LBQ[:, j], 'r-', label='Lower Bound')
    
    # Plot the contamination event simulated earlier
    axs[j].plot(Qmsx.Time / 24 / 60 / 60, Qmsx.Quality[sensor][:, 0], 'b-', label='Contamination Event')
    
    axs[j].grid(True)
    axs[j].legend(loc='upper right', fontsize=9)
    axs[j].set_ylabel('Cl_2 (mg/L)')
    axs[j].set_xlabel('Time (days)')
    axs[j].set_title(sensor_id[j])

plt.tight_layout()
plt.show()

### Contamination detection
##### Compare what the model predicts, with what the (chlorine) sensors measure. If the difference is significantly larger, then put a fault warning.    



$ \\ 
e = y - \hat{y} $

If $ e > \overline{ε} $
then fault flag $ φ(k) = 1$




In [ ]:
# Create subplots
fig, axs = plt.subplots(len(sensor_index), 1, figsize=(10, 15))

# Iterate over sensor indices
for j, sensor in enumerate(sensor_index):
    # Difference between the lower bound and the measured value
    e = LBQ[:, j] - Qmsx.Quality[sensor][:, 0]
    
    # Create the phi signal by putting 1 when the error is > 0
    phi = e > 0
    
    # Plot the results
    axs[j].plot(Qmsx.Time / 24 / 60 / 60, phi, 'r-')
    axs[j].set_title(sensor_id[j])
    axs[j].set_ylabel('φ')
    axs[j].set_xlabel('Time (days)')
    axs[j].grid(True)

plt.tight_layout()
plt.show()